# Data Loading and Preprocessing
Import pandas, numpy, and matplotlib. Load the CSV file and clean the data, handling any missing values or data type conversions.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the CSV file into a DataFrame
file_path = '/Users/ethanchung/Downloads/DATA SCIENCE/Affordable_Housing.csv'
housing_data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(housing_data.head())

# Check for missing values
print(housing_data.isnull().sum())

# Fill missing numeric values with 0 and forward-fill categorical values
housing_data.fillna({
    col: 0 if housing_data[col].dtype in ['int64', 'float64'] else None
    for col in housing_data.columns
}, inplace=True)
housing_data.fillna(method='ffill', inplace=True)

# Convert data types where necessary
housing_data['TOTAL_AFFORDABLE_UNITS'] = pd.to_numeric(housing_data['TOTAL_AFFORDABLE_UNITS'], errors='coerce')
housing_data['LATITUDE'] = pd.to_numeric(housing_data['LATITUDE'], errors='coerce')
housing_data['LONGITUDE'] = pd.to_numeric(housing_data['LONGITUDE'], errors='coerce')

# Verify data types
print(housing_data.dtypes)

# Save cleaned data for further analysis
housing_data.to_csv('/Users/ethanchung/Downloads/DATA SCIENCE/Cleaned_Affordable_Housing.csv', index=False)

# Ward-Level AMI Analysis
Group data by ward and calculate total units at different AMI levels (0-30%, 31-50%, 51-60%, 61-80%, 81%+). Create summary statistics for each ward's AMI distribution.

In [ ]:
# Group data by ward and calculate total units at different AMI levels
ward_ami_summary = housing_data.groupby('MAR_WARD').agg({
    'AFFORDABLE_UNITS_AT_0_30_AMI': 'sum',
    'AFFORDABLE_UNITS_AT_31_50_AMI': 'sum',
    'AFFORDABLE_UNITS_AT_51_60_AMI': 'sum',
    'AFFORDABLE_UNITS_AT_61_80_AMI': 'sum',
    'AFFORDABLE_UNITS_AT_81_AMI': 'sum'
}).reset_index()

# Rename columns for better readability
ward_ami_summary.rename(columns={
    'AFFORDABLE_UNITS_AT_0_30_AMI': 'Units_0_30_AMI',
    'AFFORDABLE_UNITS_AT_31_50_AMI': 'Units_31_50_AMI',
    'AFFORDABLE_UNITS_AT_51_60_AMI': 'Units_51_60_AMI',
    'AFFORDABLE_UNITS_AT_61_80_AMI': 'Units_61_80_AMI',
    'AFFORDABLE_UNITS_AT_81_AMI': 'Units_81_AMI'
}, inplace=True)

# Add a column for total affordable units per ward
ward_ami_summary['Total_Affordable_Units'] = ward_ami_summary[
    ['Units_0_30_AMI', 'Units_31_50_AMI', 'Units_51_60_AMI', 'Units_61_80_AMI', 'Units_81_AMI']
].sum(axis=1)

# Display the summary statistics
print(ward_ami_summary)

# Plot the AMI distribution for each ward
ward_ami_summary.set_index('MAR_WARD').plot(
    kind='bar',
    stacked=True,
    figsize=(12, 6),
    title='Ward-Level AMI Distribution'
)
plt.xlabel('Ward')
plt.ylabel('Number of Affordable Units')
plt.legend(title='AMI Levels')
plt.tight_layout()
plt.show()

# Affordability Distribution Analysis
Create visualizations showing the distribution of affordable units across AMI levels for each ward. Calculate percentages and ratios of affordability levels.

In [ ]:
# Calculate percentages of affordability levels for each ward
for ami_level in ['Units_0_30_AMI', 'Units_31_50_AMI', 'Units_51_60_AMI', 'Units_61_80_AMI', 'Units_81_AMI']:
    ward_ami_summary[f'{ami_level}_Percentage'] = (
        ward_ami_summary[ami_level] / ward_ami_summary['Total_Affordable_Units'] * 100
    )

# Plot the percentage distribution of AMI levels for each ward
ami_percentage_columns = [
    'Units_0_30_AMI_Percentage',
    'Units_31_50_AMI_Percentage',
    'Units_51_60_AMI_Percentage',
    'Units_61_80_AMI_Percentage',
    'Units_81_AMI_Percentage'
]

ward_ami_summary.set_index('MAR_WARD')[ami_percentage_columns].plot(
    kind='bar',
    stacked=True,
    figsize=(12, 6),
    title='Percentage Distribution of Affordable Units by AMI Levels per Ward',
    colormap='viridis'
)
plt.xlabel('Ward')
plt.ylabel('Percentage of Affordable Units')
plt.legend(title='AMI Levels')
plt.tight_layout()
plt.show()

# Save the updated summary with percentages to a CSV file
ward_ami_summary.to_csv('/Users/ethanchung/Downloads/DATA SCIENCE/Ward_AMI_Summary.csv', index=False)

# Visualization of Ward Affordability
Generate bar charts and pie charts comparing affordability levels across wards. Create stacked bar charts showing AMI distribution within each ward.

In [ ]:
# Generate bar charts comparing total affordability levels across wards
plt.figure(figsize=(10, 6))
plt.bar(ward_ami_summary['MAR_WARD'], ward_ami_summary['Total_Affordable_Units'], color='skyblue')
plt.title('Total Affordable Units by Ward')
plt.xlabel('Ward')
plt.ylabel('Total Affordable Units')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Generate pie charts for affordability levels in each ward
for index, row in ward_ami_summary.iterrows():
    plt.figure(figsize=(8, 8))
    plt.pie(
        [row['Units_0_30_AMI'], row['Units_31_50_AMI'], row['Units_51_60_AMI'], row['Units_61_80_AMI'], row['Units_81_AMI']],
        labels=['0-30% AMI', '31-50% AMI', '51-60% AMI', '61-80% AMI', '81%+ AMI'],
        autopct='%1.1f%%',
        startangle=140,
        colors=plt.cm.Paired.colors
    )
    plt.title(f'Affordability Levels in {row["MAR_WARD"]}')
    plt.tight_layout()
    plt.show()

# Create stacked bar charts showing AMI distribution within each ward
ward_ami_summary.set_index('MAR_WARD')[['Units_0_30_AMI', 'Units_31_50_AMI', 'Units_51_60_AMI', 'Units_61_80_AMI', 'Units_81_AMI']].plot(
    kind='bar',
    stacked=True,
    figsize=(12, 6),
    colormap='tab20',
    title='Stacked Bar Chart of AMI Distribution by Ward'
)
plt.xlabel('Ward')
plt.ylabel('Number of Affordable Units')
plt.legend(title='AMI Levels')
plt.tight_layout()
plt.show()

# Statistical Summary by Ward
Calculate key statistics like mean, median, and mode of affordable units per ward. Analyze the correlation between ward location and affordability levels.

In [ ]:
# Calculate statistical summaries for each ward
ward_stats = housing_data.groupby('MAR_WARD').agg({
    'TOTAL_AFFORDABLE_UNITS': ['mean', 'median', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan],
    'AFFORDABLE_UNITS_AT_0_30_AMI': 'mean',
    'AFFORDABLE_UNITS_AT_31_50_AMI': 'mean',
    'AFFORDABLE_UNITS_AT_51_60_AMI': 'mean',
    'AFFORDABLE_UNITS_AT_61_80_AMI': 'mean',
    'AFFORDABLE_UNITS_AT_81_AMI': 'mean'
}).reset_index()

# Rename columns for better readability
ward_stats.columns = [
    'MAR_WARD', 
    'Mean_Total_Units', 
    'Median_Total_Units', 
    'Mode_Total_Units', 
    'Mean_Units_0_30_AMI', 
    'Mean_Units_31_50_AMI', 
    'Mean_Units_51_60_AMI', 
    'Mean_Units_61_80_AMI', 
    'Mean_Units_81_AMI'
]

# Display the statistical summary
print(ward_stats)

# Analyze correlation between ward location and affordability levels
correlation_matrix = housing_data[['TOTAL_AFFORDABLE_UNITS', 'AFFORDABLE_UNITS_AT_0_30_AMI', 
                                   'AFFORDABLE_UNITS_AT_31_50_AMI', 'AFFORDABLE_UNITS_AT_51_60_AMI', 
                                   'AFFORDABLE_UNITS_AT_61_80_AMI', 'AFFORDABLE_UNITS_AT_81_AMI']].corr()

# Display the correlation matrix
print(correlation_matrix)

# Plot heatmap of the correlation matrix
plt.figure(figsize=(10, 8))
plt.title('Correlation Matrix of Affordability Levels')
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.tight_layout()
plt.show()

# Save the statistical summary to a CSV file
ward_stats.to_csv('/Users/ethanchung/Downloads/DATA SCIENCE/Ward_Statistical_Summary.csv', index=False)

# Geographic Analysis
Use latitude and longitude data to create geographic visualizations of affordability distribution. Plot housing locations on a map colored by AMI levels.

In [ ]:
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

# Create a GeoDataFrame from the housing data
gdf = gpd.GeoDataFrame(
    housing_data,
    geometry=gpd.points_from_xy(housing_data['LONGITUDE'], housing_data['LATITUDE']),
    crs="EPSG:4326"
)

# Initialize a Folium map centered around Washington, DC
dc_map = folium.Map(location=[38.9072, -77.0369], zoom_start=12)

# Define a color mapping for AMI levels
def get_ami_color(row):
    if row['AFFORDABLE_UNITS_AT_0_30_AMI'] > 0:
        return 'red'
    elif row['AFFORDABLE_UNITS_AT_31_50_AMI'] > 0:
        return 'orange'
    elif row['AFFORDABLE_UNITS_AT_51_60_AMI'] > 0:
        return 'yellow'
    elif row['AFFORDABLE_UNITS_AT_61_80_AMI'] > 0:
        return 'green'
    elif row['AFFORDABLE_UNITS_AT_81_AMI'] > 0:
        return 'blue'
    else:
        return 'gray'

# Add markers to the map
marker_cluster = MarkerCluster().add_to(dc_map)
for _, row in gdf.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color=get_ami_color(row),
        fill=True,
        fill_color=get_ami_color(row),
        fill_opacity=0.7,
        popup=folium.Popup(f"Address: {row['ADDRESS']}<br>"
                           f"Project Name: {row['PROJECT_NAME']}<br>"
                           f"Total Affordable Units: {row['TOTAL_AFFORDABLE_UNITS']}<br>"
                           f"0-30% AMI Units: {row['AFFORDABLE_UNITS_AT_0_30_AMI']}<br>"
                           f"31-50% AMI Units: {row['AFFORDABLE_UNITS_AT_31_50_AMI']}<br>"
                           f"51-60% AMI Units: {row['AFFORDABLE_UNITS_AT_51_60_AMI']}<br>"
                           f"61-80% AMI Units: {row['AFFORDABLE_UNITS_AT_61_80_AMI']}<br>"
                           f"81%+ AMI Units: {row['AFFORDABLE_UNITS_AT_81_AMI']}", max_width=300)
    ).add_to(marker_cluster)

# Save the map to an HTML file
map_file_path = '/Users/ethanchung/Downloads/DATA SCIENCE/DC_Affordable_Housing_Map.html'
dc_map.save(map_file_path)

# Display the map in the notebook
dc_map